In [1]:
import pandas as pd
import datetime as dt
import time
import requests

In [2]:
# Pushshift
url = "https://api.pushshift.io/reddit/search/submission?subreddit=history"

# subreddits to try:

# https://www.reddit.com/r/HistoricalWhatIf/
# https://www.reddit.com/r/history/
# https://www.reddit.com/r/HistoryNetwork/
# https://www.reddit.com/r/AskHistorians/

In [ ]:
def psquery(sub, kind = 'submission', interval = 7, q = 5, skip = 0):
    subfields = ['title', 'selftext', 'subreddit', 'created_utc', 'author', 'num_comments', 'score', 'is_self']
    
    # establish base url and stem
    roots = f"https://api.pushshift.io/reddit/search/{kind}" # also known as the "API endpoint" 
    trunk = f"{roots}?subreddit={sub}&size=100" # always pulling max of 500
    
    # instantiate empty list for temp storage
    posts = []
    
    # implement for loop with `time.sleep(2)`
    for i in range(1, q + 1):
        url = "{}&after={}d".format(trunk, ((interval * i) + skip))
        print("Querying from: " + url)
        response = requests.get(url)
        assert response.status_code == 200
        harvest = response.json()['data']
        df = pd.DataFrame.from_dict(harvest)
        posts.append(df)
        time.sleep(2)
    
    # pd.concat storage list
    full = pd.concat(posts, sort=False)
    
    # if submission
    if kind == "submission":
        # select desired columns
        full = full[subfields]
        # drop duplicates
        full.drop_duplicates(inplace = True)
        # select `is_self` == True
        full = full.loc[full['is_self'] == True]

    # create `timestamp` column
    full['timestamp'] = full["created_utc"].map(dt.date.fromtimestamp)
    
    print("Query Complete!")    
    return full 

In [ ]:
psquery(history, 2, )